In [2]:
from datascience import *

In [3]:
indicators = Table.read_table("indicators.csv")

In [4]:
indicators.where("Topic", are.equal_to("Alcohol")).group("Question").column("Question")

array(['Alcohol use among youth', 'Alcohol use before pregnancy',
       'Amount of alcohol excise tax by beverage type (beer)',
       'Amount of alcohol excise tax by beverage type (distilled spirits)',
       'Amount of alcohol excise tax by beverage type (wine)',
       'Binge drinking frequency among adults aged >= 18 years who binge drink',
       'Binge drinking intensity among adults aged >= 18 years who binge drink',
       'Binge drinking prevalence among adults aged >= 18 years',
       'Binge drinking prevalence among women aged 18-44 years',
       'Binge drinking prevalence among youth',
       'Chronic liver disease mortality',
       'Commercial host (dram shop) liability laws',
       'Heavy drinking among adults aged >= 18 years',
       'Heavy drinking among women aged 18-44 years',
       'Local authority to regulate alcohol outlet density',
       'Per capita alcohol consumption among persons aged >= 14 years'],
      dtype='<U70')

In [5]:
indicators.where("Topic", are.equal_to("Diabetes")).group("Question").column("Question")

array(['Adults with diagnosed diabetes aged >= 18 years who have taken a diabetes self-management course',
       'Amputation of a lower extremity attributable to diabetes',
       'Diabetes prevalence among women aged 18-44 years',
       'Dilated eye examination among adults aged >= 18 years with diagnosed diabetes',
       'Foot examination among adults aged >= 18 years with diagnosed diabetes',
       'Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes',
       'Hospitalization with diabetes as a listed diagnosis',
       'Influenza vaccination among noninstitutionalized adults aged 18-64 years with diagnosed diabetes',
       'Influenza vaccination among noninstitutionalized adults aged >= 65 years with diagnosed diabetes',
       'Mortality due to diabetes reported as any listed cause of death',
       'Mortality with diabetic ketoacidosis reported as any listed cause of death',
       'Pneumococcal vaccination among noninstitutionalized adu

In [6]:
topics = indicators.group("Topic").column("Topic")

In [7]:
topics

array(['Alcohol', 'Arthritis', 'Asthma', 'Cancer',
       'Cardiovascular Disease', 'Chronic Kidney Disease',
       'Chronic Obstructive Pulmonary Disease', 'Diabetes', 'Disability',
       'Immunization', 'Mental Health',
       'Nutrition, Physical Activity, and Weight Status', 'Older Adults',
       'Oral Health', 'Overarching Conditions', 'Reproductive Health',
       'Tobacco'], dtype='<U47')

In [8]:
unclean_alcohol = indicators.where("Topic", are.equal_to("Alcohol"))

In [18]:
overall_alcohol = unclean_alcohol.where("StratificationCategoryID1", are.equal_to("OVERALL"))

In [15]:
overall_diabetes = indicators.where("Question", are.equal_to("Prevalence of diagnosed diabetes among adults aged >= 18 years")).where("StratificationCategoryID1", are.equal_to("OVERALL"))
overall_diabetes_no_territories = overall_diabetes.where("LocationDesc", are.not_contained_in(["Guam", "Virgin Islands", "Puerto Rico", "United States"]))

In [16]:
# Cleaned overall diabetes in states over years
clean_oa_diabetes = overall_diabetes_no_territories.select("YearStart", "LocationAbbr", "LocationDesc", "Question", "DataValueUnit", "DataValue").where("DataValue", are.not_contained_in(["nan","", None]))
clean_oa_diabetes

YearStart,LocationAbbr,LocationDesc,Question,DataValueUnit,DataValue
2013,NE,Nebraska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,8.5
2014,NE,Nebraska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,9.2
2011,AK,Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,7.9
2015,WY,Wyoming,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,8.4
2015,GA,Georgia,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,10.7
2013,AK,Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,7.3
2015,IA,Iowa,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,7.7
2012,PA,Pennsylvania,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,10.2
2011,IN,Indiana,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,9.6
2013,NY,New York,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,10.6


In [19]:
# Cleaned overall diabetes in states over years
overall_alcohol_no_territories = overall_alcohol.where("LocationDesc", are.not_contained_in(["Guam", "Virgin Islands", "Puerto Rico", "United States"]))
clean_oa_alcohol = overall_alcohol_no_territories.select("YearStart", "LocationAbbr", "LocationDesc", "Question", "DataValueUnit", "DataValue").where("DataValue", are.not_contained_in(["nan","", None]))

In [20]:
clean_oa_alcohol

YearStart,LocationAbbr,LocationDesc,Question,DataValueUnit,DataValue
2012,DC,District of Columbia,Amount of alcohol excise tax by beverage type (beer),$,0.09
2015,DC,District of Columbia,Amount of alcohol excise tax by beverage type (beer),$,0.09
2012,OR,Oregon,Amount of alcohol excise tax by beverage type (beer),$,0.08
2012,MT,Montana,Amount of alcohol excise tax by beverage type (beer),$,0.14
2015,NE,Nebraska,Amount of alcohol excise tax by beverage type (wine),$,0.95
2012,SD,South Dakota,Amount of alcohol excise tax by beverage type (wine),$,0.93
2015,MI,Michigan,Amount of alcohol excise tax by beverage type (wine),$,0.51
2012,CO,Colorado,Amount of alcohol excise tax by beverage type (distilled ...,$,2.28
2015,CA,California,Amount of alcohol excise tax by beverage type (beer),$,0.2
2012,FL,Florida,Amount of alcohol excise tax by beverage type (beer),$,0.48


# Merge two tables based on common years and states with an observation and question and answer that we'll use to predict possible observation values

In [22]:
def match(table1, table2):
    """Find all entries in table1 (Single question) that have the same year and state as table2 (a lot of conditions)"""
    
    # Make empty table with columns we want (year, )
    matches = Table().with_columns("year", None, "state_code", None, "state_name", None, "question_of_observation", None, "observation_units", None, "observation_value", None, "predictor_question", None, "predictor_units", None, "predictor_value", None).where("year", False)
    # For every row we have with multiple questions(possible predictors), find corresponding value in table1 (prediction)
    for row in table2.rows:
        #print(row)
        # If entry from table2 has same year and state as our question, we can use it as a weight to predict
        table1_match = table1.where("YearStart", are.equal_to(row.YearStart)).where("LocationAbbr", are.equal_to(row.LocationAbbr))
        if table1_match.num_rows > 0:
            #print(table1_match) # our matched row
            matched_row = Table().with_columns("year", row.YearStart, "state_code", row.LocationAbbr, "state_name", table1_match.column("LocationDesc").item(0), "question_of_observation", table1_match.column("Question").item(0), "observation_units", table1_match.column("DataValueUnit").item(0), "observation_value", table1_match.column("DataValue").item(0), "predictor_question", row.Question, "predictor_units", row.DataValueUnit, "predictor_value", row.DataValue)
            #print(matched_row)
            matches.append(matched_row)
    return matches

intersected_table = match(clean_oa_diabetes, clean_oa_alcohol)

In [23]:
intersected_table.pivot("year", "state_name").show()

state_name,2011,2012,2013,2014,2015,2016
Alabama,14,15,15,15,14,10
Alaska,14,17,15,15,16,10
Arizona,14,17,14,15,16,10
Arkansas,15,17,15,15,16,10
California,14,17,13,15,16,10
Colorado,15,17,13,15,14,10
Connecticut,14,17,15,15,16,10
Delaware,14,17,15,15,14,10
District of Columbia,14,16,15,14,16,10
Florida,14,17,15,15,16,10


# Find out which questions each state answered in at least one year

In [24]:
def find_common_questions(intersected_table):
    """find the questions that are answered by all states"""
    unique_questions = intersected_table.group("predictor_question").column("predictor_question")
    print(unique_questions)
    # If each state the same question, then add it to a special list of questions that we can use as parameters in our parameters vector
    uniform_questions = []
    for question in unique_questions:
        for current_state in intersected_table.group("state_name").column("state_name"):
            # If our current state has at least one entry with the question, we'll use it
            state_with_question_rows = intersected_table.where("state_name", are.equal_to(current_state)).where("predictor_question", are.equal_to(question))
            if state_with_question_rows.num_rows > 0:
                #print(current_state, "has question")
                #print(state_name_table.column("year"))
                #print(state_with_question_rows.column("year"))
                #print(state_with_question_rows.num_rows)
                has_question = True
            # If not every state has the question, break out and set has_question to False
            else:
                #print(current_state, "does not have", question)
                has_question = False
                break
        if has_question:
            uniform_questions.append(question)
    return uniform_questions
uniform_questions = find_common_questions(intersected_table)
uniform_questions
table_uniform_questions = intersected_table.where("predictor_question", are.contained_in(uniform_questions))
table_uniform_questions.show()

['Alcohol use among youth' 'Alcohol use before pregnancy'
 'Amount of alcohol excise tax by beverage type (beer)'
 'Amount of alcohol excise tax by beverage type (distilled spirits)'
 'Amount of alcohol excise tax by beverage type (wine)'
 'Binge drinking frequency among adults aged >= 18 years who binge drink'
 'Binge drinking intensity among adults aged >= 18 years who binge drink'
 'Binge drinking prevalence among adults aged >= 18 years'
 'Binge drinking prevalence among women aged 18-44 years'
 'Binge drinking prevalence among youth' 'Chronic liver disease mortality'
 'Commercial host (dram shop) liability laws'
 'Heavy drinking among adults aged >= 18 years'
 'Heavy drinking among women aged 18-44 years'
 'Local authority to regulate alcohol outlet density'
 'Per capita alcohol consumption among persons aged >= 14 years']


year,state_code,state_name,question_of_observation,observation_units,observation_value,predictor_question,predictor_units,predictor_value
2013,VA,Virginia,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,9.3,Binge drinking prevalence among adults aged >= 18 years,%,16.5
2012,OH,Ohio,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,10.6,Chronic liver disease mortality,nan,1318
2014,MT,Montana,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,7.6,Per capita alcohol consumption among persons aged >= 14 ...,gallons,3.08
2015,NM,New Mexico,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,10.5,Binge drinking intensity among adults aged >= 18 years w ...,Number,7.5
2016,NE,Nebraska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,8.8,Binge drinking prevalence among women aged 18-44 years,%,19.9
2014,AL,Alabama,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,11.9,Chronic liver disease mortality,nan,683
2012,MN,Minnesota,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,7.3,Heavy drinking among women aged 18-44 years,%,6.1
2015,KY,Kentucky,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,13.4,Heavy drinking among adults aged >= 18 years,%,5.9
2012,LA,Louisiana,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,12.3,Binge drinking prevalence among adults aged >= 18 years,%,16.5
2011,GA,Georgia,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,10.2,Chronic liver disease mortality,nan,767


In [25]:
def find_complete_responses(uniform_questions_table):
    unique_years = uniform_questions_table.group("year").column("year")
    questions_per_year = uniform_questions_table.pivot("year", "predictor_question")
    questions_per_year.show()
    completely_responded_questions = []
    
    for question in questions_per_year.column("predictor_question"):
        complete_response_question = True
        question_table = questions_per_year.where("predictor_question", are.equal_to(question))
        # Iterate over years to check if all years have same count of questions and responses
        i = 0
        for year in unique_years:
            print(year)
            if i == 0:
                # Find number of values in year 1 to check if there is a question with consistent questions through all years
                num_vals_yr_1 = question_table.column(str(year)).item(0)
                print(num_vals_yr_1)
                i+=1
            else:
                if question_table.column(str(year)) != num_vals_yr_1:
                    complete_response_question = False
                    print("breaking")
                    break
        if complete_response_question:
            completely_responded_questions.append(question)
    return completely_responded_questions
responses_complete_full_table = find_complete_responses(table_uniform_questions)



predictor_question,2011,2012,2013,2014,2015,2016
Binge drinking frequency among adults aged >= 18 years w ...,102,102,102,102,102,102
Binge drinking intensity among adults aged >= 18 years w ...,102,102,102,102,102,102
Binge drinking prevalence among adults aged >= 18 years,102,102,102,102,102,102
Binge drinking prevalence among women aged 18-44 years,51,51,51,51,51,51
Chronic liver disease mortality,153,153,153,153,0,0
Commercial host (dram shop) liability laws,51,0,0,0,51,0
Heavy drinking among adults aged >= 18 years,102,102,102,102,102,102
Heavy drinking among women aged 18-44 years,50,51,50,51,51,51
Per capita alcohol consumption among persons aged >= 14 ...,0,0,0,51,0,0


2011
102
2012
2013
2014
2015
2016
2011
102
2012
2013
2014
2015
2016
2011
102
2012
2013
2014
2015
2016
2011
51
2012
2013
2014
2015
2016
2011
153
2012
2013
2014
2015
breaking
2011
51
2012
breaking
2011
102
2012
2013
2014
2015
2016
2011
50
2012
breaking
2011
0
2012
2013
2014
breaking


In [26]:
table_uniform_questions.where("predictor_question", are.contained_in(responses_complete_full_table)).to_csv("all_states_nutrition_predict_diabetes.csv")
questions_answered_all_states_predict_diabetes = table_uniform_questions.where("predictor_question", are.contained_in(responses_complete_full_table))

In [27]:
questions_answered_all_states_predict_diabetes

year,state_code,state_name,question_of_observation,observation_units,observation_value,predictor_question,predictor_units,predictor_value
2013,VA,Virginia,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,9.3,Binge drinking prevalence among adults aged >= 18 years,%,16.5
2015,NM,New Mexico,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,10.5,Binge drinking intensity among adults aged >= 18 years w ...,Number,7.5
2016,NE,Nebraska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,8.8,Binge drinking prevalence among women aged 18-44 years,%,19.9
2015,KY,Kentucky,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,13.4,Heavy drinking among adults aged >= 18 years,%,5.9
2012,LA,Louisiana,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,12.3,Binge drinking prevalence among adults aged >= 18 years,%,16.5
2013,MT,Montana,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,7.7,Binge drinking intensity among adults aged >= 18 years w ...,Number,7.7
2011,MT,Montana,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,7.1,Binge drinking prevalence among women aged 18-44 years,%,22.1
2016,AK,Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,7.5,Binge drinking frequency among adults aged >= 18 years w ...,Number,4.1
2013,IN,Indiana,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,10.1,Binge drinking frequency among adults aged >= 18 years w ...,Number,4.7
2015,WY,Wyoming,Prevalence of diagnosed diabetes among adults aged >= 18 ...,%,8.4,Heavy drinking among adults aged >= 18 years,%,6.2


In [43]:
import plotly.express as px
def to_float(value):
    return float(value)
questions_answered_complete = questions_answered_all_states_predict_diabetes.with_columns("observation_value",questions_answered_all_states_predict_diabetes.apply(to_float, "observation_value"), "predictor_value",questions_answered_all_states_predict_diabetes.apply(to_float, "predictor_value") )
px.choropleth(data_frame=questions_answered_complete.to_df(),color= "observation_value", locationmode="USA-states", locations="state_code",animation_frame="year", scope='usa')

In [29]:
def summarize_state_means(questionsDF):
    ret = Table().with_columns("state", None, "question_of_observation", None, "mean_observation", None, "predictor_question", None, "mean_predictor", None).where("state", -99999)
    for question in questionsDF.group("predictor_question").column("predictor_question"):
        for state in questionsDF.group("state_name").column("state_name"):
            state_table = questionsDF.where("predictor_question", are.equal_to(question)).where("state_name", are.equal_to(state))
            predictor_val_per_state = state_table.column("predictor_value")
            observation_val_per_state = state_table.group("observation_value").column("observation_value")
            predictor_mean = 0
            observation_mean = 0
            
            for i in predictor_val_per_state:
                predictor_mean += float(i)
            predictor_mean /= len(predictor_val_per_state)
    #         print(mean)
            
            for i in observation_val_per_state:
                observation_mean += float(i)
            observation_mean /= len(observation_val_per_state)
    #         print(mean)
            temp = Table().with_columns("state", state, "question_of_observation", questionsDF.column("question_of_observation")[0], "mean_observation", observation_mean, "predictor_question", question, "mean_predictor", predictor_mean)
            ret = ret.append(temp)
    return ret
summarized_state_means = summarize_state_means(questions_answered_all_states_predict_diabetes)
summarized_state_means.show()

state,question_of_observation,mean_observation,predictor_question,mean_predictor
Alabama,Prevalence of diagnosed diabetes among adults aged >= 18 ...,12.45,Binge drinking frequency among adults aged >= 18 years w ...,4.475
Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,7.54,Binge drinking frequency among adults aged >= 18 years w ...,4.475
Arizona,Prevalence of diagnosed diabetes among adults aged >= 18 ...,9.94,Binge drinking frequency among adults aged >= 18 years w ...,4.49167
Arkansas,Prevalence of diagnosed diabetes among adults aged >= 18 ...,11.7833,Binge drinking frequency among adults aged >= 18 years w ...,5.25
California,Prevalence of diagnosed diabetes among adults aged >= 18 ...,9.68,Binge drinking frequency among adults aged >= 18 years w ...,3.89167
Colorado,Prevalence of diagnosed diabetes among adults aged >= 18 ...,6.76667,Binge drinking frequency among adults aged >= 18 years w ...,4.025
Connecticut,Prevalence of diagnosed diabetes among adults aged >= 18 ...,8.34,Binge drinking frequency among adults aged >= 18 years w ...,4.05833
Delaware,Prevalence of diagnosed diabetes among adults aged >= 18 ...,9.85,Binge drinking frequency among adults aged >= 18 years w ...,4.30833
District of Columbia,Prevalence of diagnosed diabetes among adults aged >= 18 ...,8.46667,Binge drinking frequency among adults aged >= 18 years w ...,3.79167
Florida,Prevalence of diagnosed diabetes among adults aged >= 18 ...,10.3667,Binge drinking frequency among adults aged >= 18 years w ...,4.63333


In [31]:
import numpy as np
def sort_within_year(table, category):
    sorted_table = table.where("state", 9999999999999999999)
    for state in np.unique(table.column("state")):
        print(state)
        current_state = table.where("state", are.equal_to(state))
        sorted_current_state = current_state.sort(category)
        sorted_table = sorted_table.append(sorted_current_state)
    sorted_table.show()
    return sorted_table

sorted_question_state = sort_within_year(summarized_state_means, "predictor_question")

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming


state,question_of_observation,mean_observation,predictor_question,mean_predictor
Alabama,Prevalence of diagnosed diabetes among adults aged >= 18 ...,12.45,Binge drinking frequency among adults aged >= 18 years w ...,4.475
Alabama,Prevalence of diagnosed diabetes among adults aged >= 18 ...,12.45,Binge drinking intensity among adults aged >= 18 years w ...,7.24167
Alabama,Prevalence of diagnosed diabetes among adults aged >= 18 ...,12.45,Binge drinking prevalence among adults aged >= 18 years,12.625
Alabama,Prevalence of diagnosed diabetes among adults aged >= 18 ...,12.45,Binge drinking prevalence among women aged 18-44 years,11.5667
Alabama,Prevalence of diagnosed diabetes among adults aged >= 18 ...,12.45,Heavy drinking among adults aged >= 18 years,4.96667
Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,7.54,Binge drinking frequency among adults aged >= 18 years w ...,4.475
Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,7.54,Binge drinking intensity among adults aged >= 18 years w ...,7.73333
Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,7.54,Binge drinking prevalence among adults aged >= 18 years,19.1
Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,7.54,Binge drinking prevalence among women aged 18-44 years,20.3833
Alaska,Prevalence of diagnosed diabetes among adults aged >= 18 ...,7.54,Heavy drinking among adults aged >= 18 years,7.59167


In [32]:

designMatrix = []
observations = []
for state in sorted_question_state.group("state").column("state"):
    #temp_state_row = []
    observations = np.append(observations, sorted_question_state.where("state", are.equal_to(state)).column("mean_observation").item(0))
    temp_state_row = np.append(1, sorted_question_state.where("state", are.equal_to(state)).column("mean_predictor"))
    print(temp_state_row)
    designMatrix.append(temp_state_row)
designMatrix
print(len(observations))
print(len(designMatrix))

[ 1.          4.475       7.24166667 12.625      11.56666667  4.96666667]
[ 1.          4.475       7.73333333 19.1        20.38333333  7.59166667]
[ 1.          4.49166667  7.38333333 15.45833333 14.03333333  5.91666667]
[ 1.          5.25        8.35       13.88333333 14.4         5.24166667]
[ 1.          3.89166667  6.9        16.93333333 16.35        5.93333333]
[ 1.          4.025       6.99166667 18.80833333 20.          6.49166667]
[ 1.          4.05833333  6.91666667 17.8        18.73333333  6.16666667]
[ 1.          4.30833333  7.30833333 17.775      19.6         6.33333333]
[ 1.          3.79166667  6.43333333 23.79166667 29.          8.725     ]
[ 1.          4.63333333  7.15       16.61666667 16.53333333  6.95      ]
[ 1.          4.63333333  7.44166667 14.56666667 14.35        5.375     ]
[ 1.          4.975       8.21666667 19.675      18.6         7.70833333]
[ 1.          4.425       7.31666667 15.33333333 14.78333333  5.95      ]
[ 1.          4.325       7.63333333 2

In [39]:
from numpy import array
from numpy.linalg import inv
first_step = np.transpose(designMatrix).dot(designMatrix)
second_step = inv(first_step).dot(np.transpose(designMatrix))
third_step = second_step.dot(observations)

weights = third_step
single_state = sorted_question_state.group("state").column("state")[0]
sorted_questions = sorted_question_state.where("state", are.equal_to(single_state))\
                                               .column("predictor_question")
print(sorted_questions)
print(weights)

['Binge drinking frequency among adults aged >= 18 years who binge drink'
 'Binge drinking intensity among adults aged >= 18 years who binge drink'
 'Binge drinking prevalence among adults aged >= 18 years'
 'Binge drinking prevalence among women aged 18-44 years'
 'Heavy drinking among adults aged >= 18 years']
[ 9.47088531e+00  3.72239305e+00 -1.77366942e+00  1.93545702e-01
  1.05664284e-03 -1.07317497e+00]


In [49]:
weights_table = Table().with_columns("Predictor Questions", sorted_questions, "Weight", weights[1:])

In [123]:
weights_df.to_csv("nutrition_weight_on_diabetes_weights.csv")

# END OF CODE

In [50]:
px.bar(data_frame=weights_table.to_df(), x = "Predictor Questions", y = "Weight")